In [1]:
# Install required packages
!pip install transformers flask flask-cors pyngrok --quiet
!pip install sentence-transformers scikit-learn --quiet
!pip install sacremoses --quiet

# Load Models

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load BioGPT - lighter and faster medical model
tokenizer = AutoTokenizer.from_pretrained("microsoft/BioGPT")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/BioGPT",
    torch_dtype=torch.float16,  # Use half precision for faster inference
    low_cpu_mem_usage=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move BioGPT to GPU
print(f"🔧 Using device: {device}")
print(f"🤖 BioGPT model loaded and moved to {device}")

🔧 Using device: cuda
🤖 Med42 model loaded with automatic device mapping


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

biobert = SentenceTransformer('pritamdeka/S-BioBert-snli-multinli-stsb', device=device)

# Document Embeddings & Retriver

In [5]:
docs = []
doc_embeddings = []

In [6]:
def add_doc(text):
    # Embed the incoming doc using BioBERT
    embedding = biobert.encode([text], convert_to_tensor=True)[0].cpu().numpy()

    # Store in memory
    docs.append(text)
    doc_embeddings.append(embedding)

    print(f"✅ Added document. Total docs: {len(docs)}")

In [7]:
def find_most_similar_doc(query, top_k=1):
    if not doc_embeddings:
        return "❌ No documents in memory."

    # Embed the user query
    query_embedding = biobert.encode([query], convert_to_tensor=True)[0].cpu().numpy()

    # Compute cosine similarities
    similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

    # Get the top-k most similar docs
    top_indices = similarities.argsort()[-top_k:][::-1]

    results = []
    for idx in top_indices:
        results.append({
            "doc": docs[idx],
            "similarity": float(similarities[idx])
        })

    return results

# API

In [8]:
!pip install flask flask-cors

In [9]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

def generate_biogpt_response(prompt, max_length=100):
    """Generate medical response using BioGPT"""
    try:
        # Set pad token if not already set
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        
        # Encode the prompt
        inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
        
        # Generate response
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=len(inputs[0]) + 50,  # Add tokens to input length
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=2
            )
        
        # Decode and clean response
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Remove the original prompt from response
        if prompt in response:
            response = response.replace(prompt, "").strip()
        
        # Clean up response
        response = response.strip()
        if not response:
            response = "I understand your question about health. Please consult with a healthcare professional for personalized medical advice."
        
        return response
    except Exception as e:
        return f"I apologize, but I encountered an error processing your question. Please try rephrasing your question or consult with a healthcare professional."

@app.route('/embed-doc', methods=['POST'])
def embed_doc():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "Missing 'text' field."}), 400

    add_doc(text)
    return jsonify({"message": "✅ Document embedded.", "total_docs": len(docs)})

@app.route('/query', methods=['POST'])
def query():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "Missing 'query' field."}), 400

    results = find_most_similar_doc(query, top_k=1)
    return jsonify(results)

@app.route('/ask', methods=['POST'])
def ask_biogpt():
    """Main endpoint for WellnessGrid app - matches the expected format"""
    try:
        data = request.get_json()
        question = data.get("question", "")
        
        if not question:
            return jsonify({"error": "Missing 'question' field."}), 400
        
        # Find relevant documents
        relevant_docs = find_most_similar_doc(question, top_k=2)
        
        # Prepare context from documents
        context = ""
        sources = []
        if relevant_docs and relevant_docs != "❌ No documents in memory.":
            for doc_data in relevant_docs:
                context += f"Context: {doc_data['doc'][:200]}...\n"
                sources.append({
                    "title": "Health Document", 
                    "content": doc_data['doc'][:100] + "...",
                    "similarity": doc_data['similarity']
                })
        
        # Create medical prompt for BioGPT
        medical_prompt = f"Medical Question: {question}\n{context}Answer:"
        
        # Generate response using BioGPT
        ai_response = generate_biogpt_response(medical_prompt, max_length=150)
        
        # Return in expected format for WellnessGrid app
        return jsonify({
            "response": ai_response,
            "sources": sources,
            "mockMode": False
        })
        
    except Exception as e:
        return jsonify({
            "response": f"I apologize, but I encountered an error processing your question: {str(e)}",
            "sources": [],
            "mockMode": True
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        "status": "healthy",
        "model": "BioGPT",
        "documents_loaded": len(docs)
    })

In [10]:
!pip install pyngrok

In [12]:
# Get ngrok auth token from user
from getpass import getpass
print("🔑 Please enter your ngrok auth token (get it from https://ngrok.com)")
print("   Your token will be hidden for security")
token = getpass("Enter ngrok auth token: ")

# Setup ngrok
from pyngrok import ngrok
ngrok.set_auth_token(token)
print("✅ Ngrok token set successfully!")

# Add sample health documents for testing
print("📚 Adding sample health documents...")

sample_docs = [
    "Diabetes is a chronic condition that affects how your body processes blood glucose. Type 2 diabetes can often be managed through diet, exercise, and medication. Regular monitoring of blood sugar levels is essential.",
    "Hypertension, or high blood pressure, is often called the 'silent killer' because it usually has no symptoms. It can be managed through lifestyle changes including reducing sodium intake, regular exercise, and stress management.",
    "Depression is a mood disorder that affects how you feel, think, and handle daily activities. Treatment may include psychotherapy, medication, or a combination of both. Regular exercise and social support are also beneficial.",
    "Anxiety disorders are characterized by excessive worry or fear. Cognitive behavioral therapy (CBT) and relaxation techniques can be effective treatments along with medication when necessary.",
    "Heart disease prevention includes maintaining a healthy diet low in saturated fats, regular physical activity, not smoking, and managing stress levels."
]

for doc in sample_docs:
    add_doc(doc)

print("✅ Sample documents loaded!")

In [ ]:
# Test BioGPT model with a simple medical question
print("🧪 Testing BioGPT model...")

test_prompt = "What is diabetes?"
test_response = generate_biogpt_response(test_prompt, max_length=100)

print(f"Question: {test_prompt}")
print(f"BioGPT Response: {test_response}")
print("✅ BioGPT test completed!")


In [ ]:
# import os
# import signal
# from pyngrok import ngrok

# # Kill ngrok
# ngrok.kill()

# # Kill the Flask thread by killing the whole Colab process (if needed)
# os.kill(os.getpid(), signal.SIGKILL)

# print('Killed')

In [ ]:
# Validate everything is loaded before starting Flask
print("🔍 Validating setup...")

# Check if models are loaded
try:
    model_check = model is not None and tokenizer is not None
    biobert_check = biobert is not None
    docs_check = len(docs) > 0
    
    if not model_check:
        raise ValueError("❌ BioGPT model or tokenizer not loaded!")
    
    if not biobert_check:
        raise ValueError("❌ BioBERT model not loaded!")
    
    if not docs_check:
        raise ValueError("❌ No documents loaded!")
    
    print(f"✅ BioGPT model: Loaded")
    print(f"✅ BioBERT model: Loaded") 
    print(f"✅ Documents: {len(docs)} loaded")
    print(f"✅ Device: {device}")
    
    # Start ngrok tunnel
    print("🌐 Starting ngrok tunnel...")
    public_url = ngrok.connect(5000)
    print(f"🌍 Public URL: {public_url}")
    print("📋 Copy this URL to your WellnessGrid app configuration!")
    
    # Start Flask app
    print("🚀 Starting Flask app...")
    print("📡 Available endpoints:")
    print("  - POST /ask - Main endpoint for WellnessGrid")
    print("  - GET /health - Health check")
    print("  - POST /embed-doc - Add documents")
    print("  - POST /query - Query documents")
    print("\n⚠️  Keep this cell running to maintain the server!")
    
    app.run(host='0.0.0.0', port=5000, debug=False)
    
except Exception as e:
    print(f"❌ Setup validation failed: {str(e)}")
    print("Please run all previous cells in order before starting the Flask app.")
